In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler

# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
import jedi, parso
from coeditor.dataset import TokenizedEditDataset
from pprint import pprint


In [72]:
m = parso.parse(
"""

@dataclass(frozen=True)
class A:
    pass

"""
)
print(m.dump())

Module([
    PythonNode('decorated', [
        Decorator([
            Operator('@', (3, 0), prefix='\n\n'),
            PythonNode('atom_expr', [
                Name('dataclass', (3, 1)),
                PythonNode('trailer', [
                    Operator('(', (3, 10)),
                    PythonNode('argument', [
                        Name('frozen', (3, 11)),
                        Operator('=', (3, 17)),
                        Keyword('True', (3, 18)),
                    ]),
                    Operator(')', (3, 22)),
                ]),
            ]),
            Newline('\n', (3, 23)),
        ]),
        Class([
            Keyword('class', (4, 0)),
            Name('A', (4, 6), prefix=' '),
            Operator(':', (4, 7)),
            PythonNode('suite', [
                Newline('\n', (4, 8)),
                PythonNode('simple_stmt', [
                    Keyword('pass', (5, 4), prefix='    '),
                    Newline('\n', (5, 8)),
                ]),
          

In [3]:
from coeditor.git import get_commit_history

commits = get_commit_history(proj_root(), 20)
for c in commits[:3]:
    print(c)


CommitInfo(hash='19b037f36e74d7495cee5705bd945be6886bb6eb', parents=('673ecc5ac671bc6d340f409e3b9f6c5bdbca2098',), msg='Remove unused dependencies. - Update setup.py')
CommitInfo(hash='673ecc5ac671bc6d340f409e3b9f6c5bdbca2098', parents=('055403c96e9cae29979f8e85d86616cb9d4730ce',), msg='Simplify src dir structure.')
CommitInfo(hash='055403c96e9cae29979f8e85d86616cb9d4730ce', parents=('60f636c1e34b725122d8c58b3d5d0abe15c59bf2',), msg='Delete spot files.')


In [77]:
from coeditor.scoped_changes import edits_from_commit_history, _tlogger
from coeditor.c3problem import C3ProblemGenerator, C3ProblemTokenizer, JediUsageAnalyzer

generator = C3ProblemGenerator()

workdir = proj_root() / "../temp-1"
# subprocess.run(["rm", "-r", workdir])
_tlogger.clear()
problems = edits_from_commit_history(
    proj_root(), commits, workdir, change_processor=generator
)
print(f"{len(problems) = }")
for err, count in generator.analyzer.error_counts.items():
    print(f"({count=}): {err}")
display(_tlogger.as_dataframe())


processing commits: 100%|██████████| 19/19 [01:22<00:00,  4.35s/it]

len(problems) = 92


,name,count,avg_time,total_time
4,pre_edit_analysis,19,1.331362,25.295879
1,parse_module,193,0.033490,6.463585
5,process_change,19,0.331870,6.305531
3,post_edit_analysis,19,0.300928,5.717633
2,JModuleChange.from_modules,179,0.022400,4.009611
0,checkout,58,0.009654,0.559932


In [78]:
from coeditor.encoding import encode_diffs, _BaseTokenizer, Add_id, decode_tokens
from coeditor.c3problem import C3ProblemSimpleSplit, C3ProblemChangeDropout

prob_trans = C3ProblemChangeDropout()
encoder = C3ProblemTokenizer()
# %lprun -T output/lprof.txt -f C3ProblemTokenizer.tokenize_problem edits = [e for p in problems for e in encoder.tokenize_problem(p)]
new_probs = [p1 for p in problems for p1 in prob_trans.transform(p) if "dropout" in p1.transformations]
# print("original", SEP)
# print(decode_tokens(new_probs[0].span.original.tolist()))
# print("edit_lines", SEP)
# print(new_probs[0].edit_lines)
# print("delta", SEP)
# print(new_probs[0].span.delta)
edits = [encoder.tokenize_problem(p) for p in new_probs]

# dset = TokenizedEditDataset.from_edits(edits)
# pretty_print_dict(dset.overall_stats())
# e = edits[4]
# print(e.show())


In [79]:
print(edits[4].show())

--------------------------------------------------------------------------------
path: coeditor.ctx_change_encoder/C3ProblemGenerator.process_change
n_references: 12
total_reference_tks: 4306
project: temp-1
commit: CommitInfo(hash='f0a4cab073846109c9b3083f25a99bd2bde88b3a', parents=('18bed1bae904b1d484977a957aa4477ec2128c00',), msg='Make C3Problem more suitable for serialization.')
========Ground Truth========
<14>:<del>                 cspan = ChangedSpan(
<15>:<add>                 cspan = ChangedCodeSpan(
     <add>                     [_to_header(Modified.from_unchanged(s)) for s in ancestors],
<16>:<del>                     [Modified.from_unchanged(s) for s in ancestors],
<18>:<add>                     f_scope.path.module,

========Main Code========
      # module: coeditor.ctx_change_encoder
      class C3ProblemGenerator(ProjectChangeProcessor[C3Problem]):
          def process_change(
              self,
              pchange: JProjectChange,
              mod2usages: Mapping[

In [6]:
%%timeit -n 1 -r 2
analysis = UsageAnalysis(pyproj, add_implicit_rel_imports=True, record_type_usages=True)

9.82 s ± 26 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [34]:
%%timeit -n 1 -r 2
PythonProject.from_root(proj_root())

4.79 s ± 21.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [17]:
jproj = jedi.Project(proj_root())
jmodules = dict()
for f in proj_root().glob("src/**/*.py"):
    s = jedi.Script(path=f, project=jproj)
    jmodules[f] = s._module_node


In [3]:
jproj = jedi.Project(proj_root())
s = jedi.Script(path=proj_root() / "src/coeditor/usage_analysis.py")
s.goto(28, 9)


[<Name name='self', description='param self'>]